In [36]:
from pymongo import MongoClient
client = MongoClient()
db = client.nyt_dump
coll = db.articles

articles = []
for doc in coll.find():
    articles.append(''.join(doc['content']))

In [70]:
articles[0].encode('latin')

'\n\n\n\n\n\n\n\n\n\n\n\xe2\x80\x98Shining\xe2\x80\x99 Star: \xe2\x80\x9cDoctor Sleep,\xe2\x80\x9d Stephen King\xe2\x80\x99s much-\xc2\xadanticipated sequel to \xe2\x80\x9cThe Shining,\xe2\x80\x9d isn\xe2\x80\x99t on the hardcover list yet \xe2\x80\x94 its official release date is Sept. 24 \xe2\x80\x94 but back in 1977, \xe2\x80\x9cThe Shining\xe2\x80\x9d did spend time there on its way to becoming a cultural touchstone. (It also sold well in paperback, especially after Stanley Kubrick\xe2\x80\x99s 1980 film adaptation.)        \n\n\n\xe2\x80\x9cThe Shining\xe2\x80\x9d was King\xe2\x80\x99s third novel, and both the Book Review and the daily New York Times paid attention, though King might have wished they hadn\xe2\x80\x99t: \xe2\x80\x9cTo say that Stephen King is not an elegant writer,\xe2\x80\x9d Jack Sullivan groused in the Book Review, \xe2\x80\x9cis putting it mildly.\xe2\x80\x9d The daily\xe2\x80\x99s reviewer, Richard R. Lingeman, wasn\xe2\x80\x99t much kinder: \xe2\x80\x9cThe e

In [51]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

stopwords = stopwords.words('english')

processed = []
for a in articles:
    a = a.lower().encode('ascii', 'ignore')
    a = word_tokenize(a.translate(string.maketrans(string.punctuation, ' '*len(string.punctuation))))
    processed.append([w for w in a if w not in stopwords])
processed[0]

['shining',
 'star',
 'doctor',
 'sleep',
 'stephen',
 'kings',
 'much',
 'anticipated',
 'sequel',
 'shining',
 'isnt',
 'hardcover',
 'list',
 'yet',
 'official',
 'release',
 'date',
 'sept',
 '24',
 'back',
 '1977',
 'shining',
 'spend',
 'time',
 'way',
 'becoming',
 'cultural',
 'touchstone',
 'also',
 'sold',
 'well',
 'paperback',
 'especially',
 'stanley',
 'kubricks',
 '1980',
 'film',
 'adaptation',
 'shining',
 'kings',
 'third',
 'novel',
 'book',
 'review',
 'daily',
 'new',
 'york',
 'times',
 'paid',
 'attention',
 'though',
 'king',
 'might',
 'wished',
 'hadnt',
 'say',
 'stephen',
 'king',
 'elegant',
 'writer',
 'jack',
 'sullivan',
 'groused',
 'book',
 'review',
 'putting',
 'mildly',
 'dailys',
 'reviewer',
 'richard',
 'r',
 'lingeman',
 'wasnt',
 'much',
 'kinder',
 'evil',
 'slapdash',
 'unfocused',
 'eventually',
 'preposterous',
 'wrote',
 'king',
 'natural',
 'lacks',
 'control',
 'simply',
 'rears',
 'back',
 'lets',
 'fly',
 'fireball',
 'lot',
 'wild',
 

In [52]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

porter = PorterStemmer()
snowball = SnowballStemmer('english')
wordnet = WordNetLemmatizer()

portered = []
for a in processed:
    portered.append([porter.stem(w) for w in a])

snowballed = []
for a in processed:
    snowballed.append([snowball.stem(w) for w in a])
    
lemmad = []
for a in processed:
    lemmad.append([wordnet.lemmatize(w).encode('ascii', 'ignore') for w in a])
    

In [53]:
lemmad[0]

['shining',
 'star',
 'doctor',
 'sleep',
 'stephen',
 'king',
 'much',
 'anticipated',
 'sequel',
 'shining',
 'isnt',
 'hardcover',
 'list',
 'yet',
 'official',
 'release',
 'date',
 'sept',
 '24',
 'back',
 '1977',
 'shining',
 'spend',
 'time',
 'way',
 'becoming',
 'cultural',
 'touchstone',
 'also',
 'sold',
 'well',
 'paperback',
 'especially',
 'stanley',
 'kubrick',
 '1980',
 'film',
 'adaptation',
 'shining',
 'king',
 'third',
 'novel',
 'book',
 'review',
 'daily',
 'new',
 'york',
 'time',
 'paid',
 'attention',
 'though',
 'king',
 'might',
 'wished',
 'hadnt',
 'say',
 'stephen',
 'king',
 'elegant',
 'writer',
 'jack',
 'sullivan',
 'groused',
 'book',
 'review',
 'putting',
 'mildly',
 'daily',
 'reviewer',
 'richard',
 'r',
 'lingeman',
 'wasnt',
 'much',
 'kinder',
 'evil',
 'slapdash',
 'unfocused',
 'eventually',
 'preposterous',
 'wrote',
 'king',
 'natural',
 'lack',
 'control',
 'simply',
 'rear',
 'back',
 'let',
 'fly',
 'fireball',
 'lot',
 'wild',
 'pitch',

Stemming appears to chop off ends of words based on a predetermined algorithm, while lemmatization is only affecting words based on how they were used in the text. For example, it appears to recognize that "Shining" in this context refers to the name of a book and does not need to be reduced to its base form.

In [54]:
bag = set()
for doc in lemmad:
    for word in doc:
        bag.add(word)

In [57]:
bag = list(bag)

In [60]:
from collections import defaultdict
rev_look = defaultdict(list)
for i, word in enumerate(bag):
    rev_look[word].append(i)

In [64]:
import numpy as np

word_counts = []
for doc in lemmad:
    counts = np.zeros(len(rev_look))
    for word in doc:
        counts[rev_look[word]] += 1
    word_counts.append(counts)

word_counts = np.asarray(word_counts)

In [65]:
word_counts

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [68]:
doc_freqs = np.zeros(len(rev_look))
for word in rev_look:
    for doc in lemmad:
        if word in doc:
            doc_freqs[rev_look[word]] += 1
doc_freqs

array([ 1.,  1.,  4., ...,  1.,  1.,  1.])

In [80]:
normed = []
for doc in word_counts:
    if sum(doc) != 0:
        normed.append(doc / np.linalg.norm(doc))
normed = np.asarray(normed)

In [83]:
tf_idf = normed * np.log(1. / (doc_freqs + 1))

In [84]:
n_tfidf = []
for row in tf_idf:
    if sum(row) != 0:
        n_tfidf.append(row / np.linalg.norm(row))
n_tfidf = np.asarray(n_tfidf)

In [101]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def tokenize(a):
    a = a.lower().encode('ascii', 'ignore')
    a = word_tokenize(a.translate(
                string.maketrans(string.punctuation, ' '*len(string.punctuation))))
    return [wordnet.lemmatize(w).encode('ascii', 'ignore') for w in a]

vect = CountVectorizer(stop_words='english', tokenizer = tokenize)
sk_word_counts = vect.fit_transform(articles)

In [99]:
vect.get_feature_names()

['0',
 '000',
 '0005',
 '0008',
 '000th',
 '000years',
 '001',
 '001st',
 '005',
 '0055',
 '007',
 '0077',
 '0083',
 '01',
 '0106',
 '017',
 '0220',
 '029',
 '03',
 '033',
 '0363',
 '037',
 '040',
 '042',
 '0427',
 '044',
 '0440',
 '045',
 '0457',
 '046',
 '0468',
 '0479',
 '048',
 '050',
 '0555',
 '0568',
 '0600',
 '065',
 '0651',
 '0674',
 '0736',
 '074',
 '075',
 '076',
 '077',
 '080',
 '0800',
 '0805',
 '083',
 '085',
 '087',
 '09',
 '090',
 '091',
 '093',
 '097',
 '1',
 '10',
 '100',
 '10018',
 '1006',
 '100th',
 '101',
 '1010',
 '1014',
 '102',
 '1020',
 '1022',
 '1023',
 '103',
 '1030',
 '1036',
 '103rd',
 '104',
 '105',
 '106',
 '107',
 '107th',
 '108',
 '109',
 '10th',
 '11',
 '110',
 '1101',
 '1109',
 '110th',
 '111',
 '111th',
 '112',
 '1121',
 '113',
 '114',
 '1147',
 '115',
 '115th',
 '116',
 '1160',
 '116th',
 '117',
 '11746',
 '118',
 '119',
 '11th',
 '12',
 '120',
 '1205',
 '121',
 '1212',
 '122',
 '1226',
 '122s',
 '123',
 '124',
 '125',
 '126',
 '1260',
 '1264',
 '127

In [102]:
v_tfidf = TfidfVectorizer(stop_words = 'english', tokenizer = tokenize)
sk_tfidf = v_tfidf.fit_transform(articles)

In [105]:
print sk_tfidf[0]

  (0, 25505)	0.0432120556164
  (0, 13394)	0.0305346975172
  (0, 26641)	0.0404348868654
  (0, 28523)	0.0300881757354
  (0, 14091)	0.061890933052
  (0, 22279)	0.0603224823375
  (0, 10624)	0.0548766173001
  (0, 12608)	0.0296605268114
  (0, 28475)	0.0513519212462
  (0, 30359)	0.0449510974793
  (0, 10096)	0.0452723790208
  (0, 28849)	0.0412786686038
  (0, 14587)	0.0730083604871
  (0, 8686)	0.053316686636
  (0, 9946)	0.0577878213689
  (0, 20080)	0.0358059676403
  (0, 7076)	0.0730083604871
  (0, 1936)	0.0364676096855
  (0, 5575)	0.0358059676403
  (0, 27933)	0.060331002388
  (0, 28195)	0.0443376054942
  (0, 31320)	0.0396560781819
  (0, 5180)	0.0456041962412
  (0, 16593)	0.040037992131
  (0, 31236)	0.03264176243
  :	:
  (0, 20742)	0.061890933052
  (0, 26436)	0.0389329478703
  (0, 28846)	0.0659940447352
  (0, 7506)	0.0452723790208
  (0, 30710)	0.0199644043308
  (0, 28642)	0.0322285875783
  (0, 26696)	0.0387599604348
  (0, 8526)	0.0433634358537
  (0, 358)	0.054066623635
  (0, 478)	0.0367448741131

In [109]:
from sklearn.metrics.pairwise import linear_kernel

for i in xrange(len(articles)-1):
    j = i + 1
    print linear_kernel(sk_tfidf[i], sk_tfidf[j])
        

[[ 0.02305692]]
[[ 0.02885959]]
[[ 0.08060302]]
[[ 0.10265361]]
[[ 0.00806792]]
[[ 0.01277452]]
[[ 0.03179913]]
[[ 0.02237889]]
[[ 0.03186025]]
[[ 0.01133288]]
[[ 0.0112293]]
[[ 0.05050236]]
[[ 0.08641263]]
[[ 0.14880218]]
[[ 0.01079611]]
[[ 0.02093341]]
[[ 0.12905767]]
[[ 0.01299159]]
[[ 0.03942498]]
[[ 0.0081564]]
[[ 0.01891347]]
[[ 0.05328997]]
[[ 0.09407578]]
[[ 0.02660641]]
[[ 0.00896708]]
[[ 0.59285151]]
[[ 0.40387753]]
[[ 0.60418493]]
[[ 0.01607411]]
[[ 0.07104439]]
[[ 0.04471823]]
[[ 0.0161476]]
[[ 0.0159709]]
[[ 0.04443504]]
[[ 0.01266042]]
[[ 0.00775642]]
[[ 0.02088344]]
[[ 0.0195964]]
[[ 0.01116324]]
[[ 0.04370397]]
[[ 0.04082441]]
[[ 0.03609541]]
[[ 0.00389761]]
[[ 0.00877625]]
[[ 0.03166012]]
[[ 0.09306403]]
[[ 0.05128287]]
[[ 0.03914551]]
[[ 0.01107401]]
[[ 0.01707255]]
[[ 0.04760608]]
[[ 0.02854471]]
[[ 0.01206598]]
[[ 0.01103869]]
[[ 0.00904333]]
[[ 0.01659707]]
[[ 0.02534878]]
[[ 0.01047482]]
[[ 0.01432831]]
[[ 0.02817722]]
[[ 0.03419875]]
[[ 0.04115585]]
[[ 0.02719624